## Import Packages

In [ ]:
import cv2
import numpy as np
from dataloader import MOVE
import os
import random

## Global Variables

In [ ]:
RANDOM_SAMPLE = False

### Helper Functions

In [ ]:
def generate_path(rando : bool) -> str:
    if rando==True:
        ROOT = os.path.dirname(os.getcwd())
        DATA_DIR = os.path.join(ROOT, 'data')
        FIN_DIR = os.path.join(DATA_DIR, random.choice(list(filter(lambda x: os.path.isdir(os.path.join(DATA_DIR, x)), os.listdir(DATA_DIR)))))
        vp = os.path.join(FIN_DIR, random.choice(list(filter(lambda x: os.path.splitext(x)[1]=='.mov', os.listdir(FIN_DIR)))))
    else:
        vp = r'..\data\TH16\TH16ST25A2D1000TS0_2.mov'

    print(vp)

    return vp

## Dynamic Feature Detection
Pre-processing includes four steps
1. We need to load our *.mov* file in as our MOVE object. 
2. We convert to grayscale to reduce the information to process (and remove color variation)
3. Then, in preparation to "remove" the background from the video, we need to reduce the contrast
4. Lastly we need to remove the background

In [ ]:
vid_path = generate_path(False)
recording = MOVE(vid_path)
recording.play_video( )
recording.to_gray(inplace=True)


In [ ]:
recording.change_contrast(alpha=0.5, beta=2, inplace=True)
recording.remove_avg(skipframes=100)

In [ ]:
recording.play_video()

Here, we will create a copy (because we were too lazy to create an undo feature) to act as a checkpoint. We can optionally use a morphological transformation to try to fill out the air pockets as much as possible. We also check our work here at the end of this. We now have our mask for our dynamic components

In [ ]:
out:MOVE = recording.ranger(minval=100, maxval=240)
out.closing(inplace=True, ksize=(5,5), iterations=1)

### Error Checking

We can now check our mask

In [ ]:
out.play_video()

To ensure the accuracy of our mask, we will use one of the best, most robust vision systems: the human eye. We will overlay our mask to highlight the masked areas to a brighter level than the non-masked area. This is not super effective for spotting holes in bubble masks, but it is great for spotting missed bubbles.

In [ ]:
rec1 = MOVE(vid_path)
rec1.apply_mask(out, inplace=True)
rec1.play_video()

## Waterline Detection

In [ ]:
recording = MOVE(vid_path)
recording.to_gray(inplace=True)
# vid_path = generate_path(True)

In [ ]:
recording.play_video()

In [ ]:
background = recording.get_avg()
background = cv2.GaussianBlur(background, (1111, 1), sigmaX=32000, borderType=cv2.BORDER_REFLECT)
MOVE.show(background)

In [ ]:
Z = np.float32(background.reshape((-1, 3)))

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
_, labels, centers = cv2.kmeans(Z, 3, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = np.uint8(centers)
quantized = centers[labels.flatten()]
quantized = quantized.reshape(background.shape)
MOVE.show(quantized)

In [ ]:
yMin = quantized.shape[1]//4
darkestColor = np.unique(quantized).min()

half_mask:np.ndarray = np.ones_like(quantized)
half_mask[:yMin, :] = 0

In [ ]:
waterline = 255*np.where(quantized[:, ] == darkestColor, half_mask, np.zeros_like(recording.frames[0]))
MOVE.show(waterline)

In [ ]:
rec2 = MOVE(vid_path)
temp = MOVE(vid_path)
temp.img_like(img=waterline, inplace=True)
rec2.apply_mask(temp, inplace=True)
rec2.play_video()

In [ ]:
temp.combine_masks(out, inplace=True)
temp.play_video()

In [ ]:
for i in range(len(temp.frames)):
    final_frame = temp.frames[i]
    max_bright = final_frame.max()
    final_frame = final_frame.T
    for r, row in enumerate(final_frame):
        min_ind = np.where(row == max_bright)[0][0]
        final_frame[r][:min_ind] = 0

    temp.frames[i] = final_frame.T

temp.play_video()